In [1]:
import requests
import time

In [2]:
query = "perovskite solar halide passivation"
base_url = f"https://api.crossref.org/works"

rows_per_request = 1000
offset = 0
all_dois = []

while True:
    url = f"{base_url}?filter=from-pub-date:2020&query={query}&rows={rows_per_request}&offset={offset}"
    response = requests.get(url)

    if response.status_code != 200:
        print(f"Failed request at offset {offset}: {response.status_code}")
        break

    data = response.json()
    items = data['message']['items']

    if not items:  # Stop when there are no more results
        break

    # Extract DOIs
    for item in items:
        if 'DOI' in item:
            all_dois.append(item)

    print(f"Fetched {len(items)} records (Offset: {offset})")
    offset += rows_per_request  # Move to the next batch
    time.sleep(1)  # Be polite and avoid rate limits

Fetched 1000 records (Offset: 0)
Fetched 1000 records (Offset: 1000)
Fetched 1000 records (Offset: 2000)
Fetched 1000 records (Offset: 3000)
Fetched 1000 records (Offset: 4000)
Fetched 1000 records (Offset: 5000)
Fetched 1000 records (Offset: 6000)
Fetched 1000 records (Offset: 7000)
Fetched 1000 records (Offset: 8000)
Fetched 1000 records (Offset: 9000)
Failed request at offset 10000: 400


In [24]:
len(all_dois)

25000

In [3]:
import numpy as np

np.mean([True if 'abstract' in doi else False for doi in all_dois])

0.3528